## LLM Motivation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//velebit-ai/research-llm-development/blob/master/Development_with_LLM_quick_intro.ipynb)

**Why are we here?**

- **The paradigm has changed** how we approach machine learning projects
- **Old way**
    - collect labeled data
    - train a supervised machine learning model (most common)
    - It takes time, effort, and expertize
- **New way**
    - start with few-shot and zero-shot learning (i.e. prompt engineering)
    - try something fast with an API or a free LLM
    - do the old way later on when you see you need it
    - LLMs augmented by external knowledge and tools
    - but, a lot of messy engineering effort for the last mile
    - lack of exact control
- **Types of model training tasks**
    - next word prediction on unlabeled texts
        - Internet scale, costly
        - base model development
    - supervised fine-tuning (instruct and chat models)
        - collect prompt-response pairs
        - common and efficient
        - also next word prediction, but on prepared data
        - gets the "autistic" base model to show its knowledge
    - RLHF (and similar)
        - reinforcement learning addition
        - collect triplets (prompt, response, better response)
        - use comparison data to further improve on supervised fine-tuning
        - more complex to do
        - often supervised fine-tuning good enough

**Some references**

[State of GPT, Andrej Karpathy](https://www.youtube.com/watch?v=bZQun8Y4L2A&ab_channel=MicrosoftDeveloper)

[Hacker's guide to LLMs](https://www.youtube.com/watch?v=jkrNMKz9pWU&ab_channel=JeremyHoward)

[Deeplearning.ai short LLM courses](https://www.deeplearning.ai/short-courses/)

[RLHF basics](https://huggingface.co/blog/rlhf)


## Quick intro into OpenAI API

**Credits**

Examples from Shawhin Talebi's [blog](https://towardsdatascience.com/cracking-open-the-openai-python-api-230e4cae7971) and [code](https://github.com/ShawhinT/YouTube-Blog/blob/main/LLMs/openai-api/openai-api-demo.ipynb), commit `632d2610a9fe7f628b77a61a0744080c0113a64d`

### set-up

In [1]:
! pip install openai watermark -q

In [2]:
import openai
import time
import watermark

In [3]:
%load_ext watermark

In [4]:
%watermark

Last updated: 2023-10-10T04:50:54.767030+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.120+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [5]:
%watermark -iv

watermark: 2.4.3
openai   : 0.28.1



In [6]:
# Use your OpenAI API key

openai.api_key = "your key"

### Some OpenAI API calls

##### First call

In [7]:
# create a chat completion
chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                messages=[{"role": "user", "content": "Listen to your"}])

In [8]:
chat_completion.to_dict()

{'id': 'chatcmpl-87yznuXXFThb5SHGb5j4jye8zq99t',
 'object': 'chat.completion',
 'created': 1696913455,
 'model': 'gpt-3.5-turbo-0613',
 'choices': [<OpenAIObject at 0x7fa392978e00> JSON: {
    "index": 0,
    "message": {
      "role": "assistant",
      "content": "heart. It holds the truth that your mind may try to question or ignore. Your heart knows what truly makes you happy, what fulfills you, and what brings you joy. Trust in its wisdom and guidance, even if it means going against rationality or societal expectations. Your heart knows the path that is right for you, so listen to it and follow its lead."
    },
    "finish_reason": "stop"
  }],
 'usage': <OpenAIObject at 0x7fa3799abb00> JSON: {
   "prompt_tokens": 10,
   "completion_tokens": 75,
   "total_tokens": 85
 }}

In [9]:
# print the chat completion
print(chat_completion.choices[0].message.content)

heart. It holds the truth that your mind may try to question or ignore. Your heart knows what truly makes you happy, what fulfills you, and what brings you joy. Trust in its wisdom and guidance, even if it means going against rationality or societal expectations. Your heart knows the path that is right for you, so listen to it and follow its lead.


##### max tokens

In [10]:
# create a chat completion
chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                messages=[{"role": "user", "content": "Listen to your"}],
                                max_tokens = 1)

# print the chat completion
print(chat_completion.choices[0].message.content)

heart


##### n = number of chat completions

In [11]:
# create a chat completion
chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                messages=[{"role": "user", "content": "Listen to your"}],
                                max_tokens = 2,
                                n=5)

# print the chat completion
for i in range(len(chat_completion.choices)):
    print(chat_completion.choices[i].message.content)

inner voice
intuition
intuition
heart.
heart.


##### temperature

In [12]:
# create a chat completion
chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                messages=[{"role": "user", "content": "Listen to your"}],
                                max_tokens = 2,
                                n=5,
                                temperature=0)

# print the chat completion
for i in range(len(chat_completion.choices)):
    print(chat_completion.choices[i].message.content)

heart.
heart.
heart.
heart.
heart.


In [13]:
# create a chat completion
chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                messages=[{"role": "user", "content": "Listen to your"}],
                                max_tokens = 2,
                                n=5,
                                temperature=2)

# print the chat completion
for i in range(len(chat_completion.choices)):
    print(chat_completion.choices[i].message.content)

uisupported
inner voice
heart birthday
broª
MPF


### Demo: Lyric Completion Assistant

In [14]:
# initial prompt with system message and 2 task examples
messages_list = [{"role":"system", "content": "I am Roxette lyric completion assistant. When given a line from a song, I will provide the next line in the song."},
                 {"role":"user", "content": "I know there's something in the wake of your smile"},
                 {"role":"assistant", "content": "I get a notion from the look in your eyes, yeah"},
                 {"role":"user", "content": "You've built a love but that love falls apart"},
                 {"role":"assistant", "content": "Your little piece of Heaven turns too dark"},
                 {"role":"user", "content": "Listen to your"}]

In [15]:
for i in range(4):
    # create a chat completion
    chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                    messages=messages_list,
                                    max_tokens = 15,
                                    n=1,
                                    temperature=0)

    # print the chat completion
    print(chat_completion.choices[0].message.content)

    new_message = {"role":"assistant", "content":chat_completion.choices[0].message.content} # append new message to message list
    messages_list.append(new_message)
    time.sleep(0.1)

Heart when he's calling for you
Listen to your heart, there's nothing else you can do
I don't know where you're going and I don't know why
But listen to your heart before you tell him goodbye


In [16]:
# Actual lyrics:

# Listen to your heart when he's calling for you
# Listen to your heart, there's nothing else you can do
# I don't know where you're going and I don't know why
# But listen to your heart before you tell him goodbye

## Prompt Engineering

**Some common tips from the community**

- Be Descriptive
- Give Examples
- Be clear and precise
- Like talking to an expert friend
- Use structured inputs
- Let's think step by step
- Use Chatbot personas
- Encourage the LLM to ask questions
- Ask to explain reasoning

**Many resources out there. For example:**

[OpenAI CookBook](https://cookbook.openai.com/)

[Prompt Engineering Guide](https://github.com/dair-ai/Prompt-Engineering-Guide)
